In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
from datetime import datetime, timedelta

import rqdatac as rq
import alphalens as al


## Single Factor Testing using Alphalens

#### Sector Classification

In [ ]:
# MSCI全球标准行业分类
sectors = ['Energy','Materials','ConsumerDiscretionary','ConsumerStaples',
          'HealthCare','Financials','RealEstate','InformationTechnology',
          'TelecommunicationServices','Utilities','Industrials']
sector_names = {'Energy':'能源','Materials':'原材料','ConsumerDiscretionary':'非必须消费品',
               'ConsumerStaples':'必需消费品','HealthCare':'医疗保健','Financials':'金融',
               'RealEstate':'房地产','InformationTechnology':'信息技术',
                'TelecommunicationServices':'电信服务','Utilities':'公共服务',
                'Industrials':'工业'}

sector_dict = {sector:rq.sector(sector) for sector in sectors}
code_dict = {}
for k, v in sector_dict.items():
    for code in v:
        code_dict[code] = k

#### Import price data and factor data

In [ ]:
backtest_price_data = pd.read_hdf('10-year non-ST price data.h5')
backtest_price_data = backtest_price_data.fillna(method='ffill')

In [ ]:
roe = pd.read_hdf('roe.h5').reset_index().set_index(['date','order_book_id']).fillna(method='ffill')

In [ ]:
pb = pd.read_hdf('pb.h5').reset_index().set_index(['date','order_book_id']).fillna(method='ffill')

In [ ]:
mkt_cap = pd.read_hdf('mkt_cap.h5').reset_index().set_index(['date','order_book_id']).fillna(method='ffill')

In [ ]:
vol3 = pd.read_hdf('vol3.h5').reset_index().set_index(['date','order_book_id']).fillna(method='ffill')

In [ ]:
stock_codes = list(set(roe.index.get_level_values(1)))
backtest_price_data = backtest_price_data[stock_codes]
ticker_sector = {key: value for key, value in code_dict.items() if key in stock_codes}

In [ ]:
pb_factor_data = al.utils.get_clean_factor_and_forward_returns(factor=pb,
                                                               prices=backtest_price_data,
                                                               periods=(1,5,10),
                                                               max_loss=0.4,
                                                               groupby_labels=sector_names,
                                                               groupby=ticker_sector)

In [ ]:
roe_factor_data = al.utils.get_clean_factor_and_forward_returns(factor=roe,
                                                               prices=backtest_price_data,
                                                               periods=(1,5,10), max_loss=0.4)

In [ ]:
mkt_cap_factor_data = al.utils.get_clean_factor_and_forward_returns(factor=mkt_cap,
                                                                   prices=backtest_price_data,
                                                                   periods=(1,5,10), max_loss=0.4)

In [ ]:
vol3_factor_data = al.utils.get_clean_factor_and_forward_returns(factor=vol3,
                                                                   prices=backtest_price_data,
                                                                   periods=(1,5,10), max_loss=0.4)

In [ ]:
al.tears.create_returns_tear_sheet(factor_data=roe_factor_data)

In [ ]:
al.tears.create_returns_tear_sheet(factor_data=mkt_cap_factor_data)

In [ ]:
al.tears.create_returns_tear_sheet(factor_data=vol3_factor_data)

In [ ]:
al.tears.create_information_tear_sheet(factor_data=roe_factor_data)

In [ ]:
al.tears.create_turnover_tear_sheet(factor_data=roe_factor_data,
                                   turnover_periods=['1D','5D','10D'])

In [ ]:
al.tears.create_returns_tear_sheet(factor_data=pb_factor_data)

In [ ]:
al.tears.create_information_tear_sheet(factor_data=pb_factor_data)

In [ ]:
al.tears.create_turnover_tear_sheet(factor_data=pb_factor_data,turnover_periods=['1D','5D','10D'])